In [1]:
from tg.grammar_ru import *
import numpy as np

FOLDER = Loc.data_cache_path/'Martin'
librusec_path = FOLDER/'50K_decrease_fix'/'matrix.npy'
martin_path = FOLDER/'50K_Martin_on_Martin'/'Martin_cooc'
combo_path = FOLDER/'comb_cooc_2d_fixed'

In [2]:
import pandas as pd

tuning_candidates = pd.read_csv(FOLDER/'tuning_candidates.csv', sep='\t').set_index('nf_n')
all_freq_nnf = tuning_candidates[tuning_candidates.in_freq_prop | 
                                 tuning_candidates.is_name_by_cnt].index.values
#tuning_candidates.is_nonce_by_freq
#del tuning_candidates

In [ ]:
all_freq_nnf

In [3]:
LEN = 22724

In [4]:
lrc = np.load(librusec_path)
with open(combo_path, 'wb') as f:
    np.save(f, lrc.reshape(LEN, LEN))
del lrc

In [5]:
row_mask = np.array([[(x in all_freq_nnf) for x in range(LEN)]])
huge_mask = np.repeat(row_mask, repeats=LEN, axis=0)
huge_mask[all_freq_nnf] = [True]

In [6]:
### Div librusec 
combo = np.load(combo_path)
combo_sum = sum(combo[~huge_mask])
combo /= combo_sum
with open(combo_path, 'wb') as f:
    np.save(f, combo)
print(combo[3][3])
del combo

4.940652266795022e-09


In [7]:
### Get Martin divisor
author = np.load(martin_path)
author_sum = sum(author[huge_mask])
print(author[3][3] / author_sum)
del author
del huge_mask

2.142873344850158e-06


In [8]:
from tqdm import tqdm

parts = 100
size = LEN // parts
for i in tqdm(range(parts + 1)):
    start = i * size
    end = (i + 1) * size
    if i < parts:
        part_mask = np.repeat(row_mask, repeats=size, axis=0)
    else:
        part_mask = np.repeat(row_mask, repeats=(LEN % parts), axis=0)
    nonce_rows = [(x - start) for x in range(start, end) if x in all_freq_nnf]
    part_mask[nonce_rows] = [True]
    del nonce_rows
    
    author = np.load(martin_path)
    author_rows = author[start:end]
    author_rows /= author_sum
    masked = author_rows[part_mask]
    del author
    del author_rows
    
    combo = np.load(combo_path)
    combo_rows = combo[start:end]
    combo_rows[part_mask] = masked
    combo[start:end] = combo_rows
    with open(combo_path, 'wb') as f:
        np.save(f, combo)
    del combo

100%|██████████████████████████████████████████████████████████████████████| 101/101 [2:36:25<00:00, 92.93s/it]


In [3]:
combo = np.load(combo_path)
# with open(FOLDER/'comb_cooc_fixed_flt', 'wb') as f:
#     np.save(f, combo.flatten())

In [10]:
#combo.flatten()

In [11]:
#combo_fl = np.load(FOLDER/'comb_cooc_fixed_flt')

In [4]:
from glove_helpers.prep import MatrixCollector
from glove_helpers.training import Training
from yo_fluq_ds import *

Training.write_cooc(
    MatrixCollector.matrix_to_glove_tuples(combo.flatten(), 22724).feed(fluq.with_progress_bar()),
    FOLDER/'cooc_ifp_inbc_mixed'
)

  0%|          | 0/157442608 [00:00<?, ?it/s]